In [394]:
import pandas as pd
import os
import csv

#hopefully I've joined the paths correctly
dir = os.path.abspath('/Users/maxkrueger/Documents/nu-chi-data-pt-06-2021-u-c/04-Pandas/Homework/due_July21/Instructions/HeroesOfPymoli')
datapath = os.path.join(dir,'Resources/purchase_data.csv')

reader = pd.read_csv(datapath)

In [395]:
#remove duplicates to get count
duplicates_dropped = reader.drop_duplicates(subset=['SN'])
play_num = duplicates_dropped['SN'].size
player_df = pd.DataFrame({'Number of Players': play_num}, index=['='])
player_df.style

,Number of Players
=,576


In [396]:
unique_items = reader.drop_duplicates(subset=['Item ID'])
item_num = unique_items['Item ID'].size
avg_price = unique_items['Price'].mean()
avg_spent = reader['Price'].mean()
purch_count = reader['Price'].size
total_spent = reader['Price'].sum()
summary_df = pd.DataFrame({'Number of Items': item_num, 'Average Price of Items': '${:0,.2f}'.format(avg_price), 'Average Amount Spent': '${:0,.2f}'.format(avg_spent),'Number of Purchases': purch_count, 'Total Revenue': '${:0,.2f}'.format(total_spent)},index=[0])
summary_df.style


,Number of Items,Average Price of Items,Average Amount Spent,Number of Purchases,Total Revenue
0,179,$3.04,$3.05,780,"$2,379.77"


In [397]:
gender_count = duplicates_dropped['Gender'].value_counts()
male_count = gender_count['Male']
male_perc = male_count / play_num
fem_count = gender_count['Female']
fem_perc = fem_count / play_num
other_count = gender_count['Other / Non-Disclosed']
other_perc = other_count / play_num
d = {'Total Counts': pd.Series([male_count,fem_count,other_count], index = ['Male','Female','Other/Non-Disclosed']),
'Percentage of Players': pd.Series(["{:.2%}".format(male_perc),"{:.2%}".format(fem_perc),"{:.2%}".format(other_perc)],index = ['Male','Female','Other/Non-Disclosed'])
}
gender_summary = pd.DataFrame(d)
gender_summary.index.name = "Gender"
gender_summary.style


,Total Counts,Percentage of Players
Gender,,
Male,484,84.03%
Female,81,14.06%
Other/Non-Disclosed,11,1.91%


In [398]:
ind = ['Male','Female','Other/Non-Disclosed']
male_filter = reader.loc[reader['Gender'] == 'Male']
female_filter = reader.loc[reader['Gender'] == 'Female']
other_filter = reader.loc[reader['Gender'] == 'Other / Non-Disclosed']


purch_count = [
    male_filter['Purchase ID'].size, 
    female_filter['Purchase ID'].size,
    other_filter['Purchase ID'].size
]

avg_price = [
    '${:0,.2f}'.format(male_filter['Price'].mean()),
    '${:0,.2f}'.format(female_filter['Price'].mean()),
    '${:0,.2f}'.format(other_filter['Price'].mean())
]

total_purch = [
    '${:0,.2f}'.format(male_filter['Price'].sum()),
    '${:0,.2f}'.format(female_filter['Price'].sum()),
    '${:0,.2f}'.format(other_filter['Price'].sum())
]


male_dict = {}
male_dropped = male_filter.drop_duplicates(subset=['SN'])
for sn in male_dropped['SN']:
    malesn_filter = male_filter.loc[reader['SN'] == sn]
    male_dict[sn] = malesn_filter['Price'].sum()

male_total_mu = '${:0,.2f}'.format(sum(male_dict.values())/len(male_dict))

female_dict = {}
female_dropped = female_filter.drop_duplicates(subset=['SN'])
for sn in female_dropped['SN']:
    femalesn_filter = female_filter.loc[reader['SN'] == sn]
    female_dict[sn] = femalesn_filter['Price'].sum()

female_total_mu = '${:0,.2f}'.format(sum(female_dict.values())/len(female_dict))

other_dict = {}
other_dropped = other_filter.drop_duplicates(subset=['SN'])
for sn in other_dropped['SN']:
    othersn_filter = other_filter.loc[reader['SN'] == sn]
    other_dict[sn] = othersn_filter['Price'].sum()

other_total_mu = '${:0,.2f}'.format(sum(other_dict.values())/len(other_dict))

avg_totals = [male_total_mu,female_total_mu,other_total_mu]

df = {'Purchase Count': pd.Series(purch_count, index = ind),
      'Average Purchase Price': pd.Series(avg_price, index = ind),
      'Total Purchase Value': pd.Series(total_purch, index = ind),
      'Average Total Purchase per Person': pd.Series(avg_totals, index = ind)
}
price_bygender = pd.DataFrame(df)
price_bygender.index.name = "Gender"
price_bygender.style


,Purchase Count,Average Purchase Price,Total Purchase Value,Average Total Purchase per Person
Gender,,,,
Male,652,$3.02,"$1,967.64",$4.07
Female,113,$3.20,$361.94,$4.47
Other/Non-Disclosed,15,$3.35,$50.19,$4.56


In [399]:
age_bins = [10,15,20,25,30,35,40]
ind = ['<10','10-14','15-19','20-24','25-29','30-34','35-39','40+']
age_counts = {}
age_counts[ind[0]] = duplicates_dropped.loc[duplicates_dropped['Age'] < age_bins[0]]
for i in range(1,7):
    age_counts[ind[i]] = duplicates_dropped.loc[(duplicates_dropped['Age'] >= age_bins[i-1]) & (duplicates_dropped['Age'] < age_bins[i])]
age_counts[ind[7]] = duplicates_dropped.loc[duplicates_dropped['Age'] >= age_bins[6]]

ages = {}
for i in ind:
    ages[i] = age_counts[i]['Age'].size

percs = {}
for i in ind:
    percs[i] = "{:.2%}".format(ages[i] / sum(ages.values()))

df = {'Total Count': pd.Series(ages.values(),index=ages.keys()), 'Percentage of Players':pd.Series(percs.values(),index=percs.keys())}
age_summary = pd.DataFrame(df)
age_summary.index.name= 'Age Ranges'
age_summary.style
    

,Total Count,Percentage of Players
Age Ranges,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


In [400]:
ages_wdups = {}
ages_wdups[ind[0]] = reader.loc[reader['Age'] < age_bins[0]]
for i in range(1,7):
    ages_wdups[ind[i]] = reader.loc[(reader['Age'] >= age_bins[i-1]) & (reader['Age'] < age_bins[i])]
ages_wdups[ind[7]] = reader.loc[reader['Age'] >= age_bins[6]]

purch_count = []
for i in ind:
    purch_count.append(ages_wdups[i]['Purchase ID'].size)

avg_price = []
for i in ind:
    avg_price.append('${:0,.2f}'.format(ages_wdups[i]['Price'].mean()))
    
tot_purch = []
for i in ind:
    tot_purch.append('${:0,.2f}'.format(ages_wdups[i]['Price'].sum()))

age_dict = []
for i in ind:
    sn_dict = {}
    for sn in ages_wdups[i]['SN']:
        agesn_filter = ages_wdups[i].loc[ages_wdups[i]['SN'] == sn]
        sn_dict[sn] = agesn_filter['Price'].sum()
    age_dict.append(sn_dict)

avg_tot = []
for i in range(len(age_dict)):
    avg_tot.append('${:0,.2f}'.format(sum(age_dict[i].values()) / len(age_dict[i])))

df = pd.DataFrame({'Purchase Count':purch_count,'Average Purchase Price': avg_price,'Total Purchase Value': tot_purch,'Avg Total Purchase per Person': avg_tot},index=ind)
df.index.name = 'Age Ranges'
df.style

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Ranges,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


In [401]:
sn_count = reader['SN'].value_counts()
sns = sn_count.index.tolist()

purchs = {sns[i]: (sn_count[i],'${:0,.2f}'.format(reader.loc[reader['SN'] == sns[i]]['Price'].sum() / sn_count[i])) for i in range(len(sns))}

top_purchs = {sn: purchs[sn][1] for sn in sns}

tot_purchs = [(sn, reader['Price'].loc[reader['SN'] == sn].sum()) for sn in top_purchs.keys()]
tot_purchs.sort(key = lambda x: x[1],reverse=True)
tot_purchs = {tot_purchs[i][0]: '${:0,.2f}'.format(tot_purchs[i][1]) for i in range(len(tot_purchs))}

purchs_count = {sn: reader['SN'].loc[reader['SN'] == sn].value_counts()[0] for sn in top_purchs.keys()}

df = pd.DataFrame([purchs_count, top_purchs, tot_purchs],index = ['Purchase Count','Average Purchase Price','Total Purchase Value']).T
ind_order = list(tot_purchs.keys())
df2 = df.reindex(ind_order)
df2.index.name = 'SN'
df2.head().style


,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In [402]:
id_name_price = reader[['Item ID','Item Name','Price']]
inp = id_name_price
indiv_list = reader[['Item ID','Item Name']].value_counts().index.tolist()

purch_count = [inp['Item Name'].loc[inp['Item Name']==indiv_list[i][1]].count() for i in range(len(indiv_list))]
price = ['${:0,.2f}'.format(inp['Price'].loc[inp['Item Name']==indiv_list[i][1]].sum()/purch_count[i]) for i in range(len(indiv_list))]
tot_purch = ['${:0,.2f}'.format(inp['Price'].loc[inp['Item Name']==indiv_list[i][1]].sum()) for i in range(len(indiv_list))]
ids = [inp['Item ID'].drop_duplicates().loc[inp['Item Name']==indiv_list[i][1]].item() for i in range(len(indiv_list))]

id_name = pd.MultiIndex.from_tuples([(ids[i],indiv_list[i][1]) for i in range(len(indiv_list))])

sum_dict= {(ids[i],indiv_list[i][1]): [purch_count[i],price[i],tot_purch[i]] for i in range(len(indiv_list))}
df = pd.DataFrame(sum_dict,index = ['Purchase Count','Item Price','Total Purchase Value']).T
df.index.names = ('Item ID','Item Name')

df.head().style


,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


In [403]:
df['Total Purchase Value'] = df['Total Purchase Value'].str.strip('$')
df['Total Purchase Value'] = df['Total Purchase Value'].astype(float)
df2 = df.sort_values(['Total Purchase Value'],ascending=False)
df2['Total Purchase Value'] = df['Total Purchase Value'].map('${:,.2f}'.format)
df2.head().style

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
